In [19]:
import sys
import re 
import imp
from datetime import datetime
from math import pi
import requests
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
import googlemaps
import geopandas as gpd
import json
import seaborn
from shapely.geometry import Point, Polygon
import warnings
from IPython.display import clear_output
from bs4 import BeautifulSoup
import bokeh
from bokeh.io import output_notebook, show, output_file
from bokeh.plotting import figure, gmap, output_file
from bokeh.palettes import brewer, mpl
from bokeh.models import GeoJSONDataSource,LinearColorMapper, ColorBar, Label, HoverTool, ColumnDataSource
from bokeh.models import Range1d, ColorMapper, GMapOptions
from bokeh.models.glyphs import MultiLine
from bokeh.models.annotations import Title 
from bokeh.models.callbacks import CustomJS 
from bokeh.models.widgets.inputs import MultiSelect
from bokeh.models.formatters import NumeralTickFormatter, DatetimeTickFormatter
from bokeh.tile_providers import CARTODBPOSITRON, get_provider
from bokeh.layouts import gridplot,row, column , grid
sys.setrecursionlimit(40000)
warnings.filterwarnings('ignore')
# GOOGLE API KEY FOR MAP 
file_api = '../00_Credentials_files/app_keys.py'
keys = imp.load_source('keys', file_api) 

In [20]:
# GET NEW DATA
### archived_daily_case_updates 
# def corona_data(url1, url2, filename, save = ['Y', 'N']):
#     '''
#     No longer works as Github change the way html codes for the tables
#     '''
#     dfdly = pd.read_html(url1)[0]#[2:-2]
#     dfdly = dfdly[dfdly.Name.str.contains(r'.csv', case = False)]
#     try:
#         fdf = pd.read_csv(filename, low_memory = False)
#         current_files = fdf.source.unique()
#     except:
#         fdf = pd.DataFrame()
#         current_files = []
#     counter = 0
#     for file in dfdly.Name.unique():
#         if file not in current_files:
#             counter += 1
#             clear_output(wait=True)
#             print('{} New Data Found {}'.format(counter, file)) 
#             df2 = pd.read_html(url2+file, index_col=0)[0]
#             df2.loc[:,'source'] = file
#             fdf = pd.concat([fdf, df2], 0, sort = False)     
#         else:
#             continue
#     if counter == 0:
#         print('No new updates found!!!')
#     else:
#         print('{} New updates found!!!'.format(counter))
#     if save == 'Y':
#         fdf.to_csv(filename, index = False)    
#         print('SAVED')
#     return fdf

# ## archived_daily_case_updates 
# # url1 = 'https://github.com/CSSEGISandData/COVID-19/tree/master/archived_data/archived_daily_case_updates'
# # url2 = 'https://github.com/CSSEGISandData/COVID-19/blob/master/archived_data/archived_daily_case_updates/'
# # file1 = 'coronavirus_dly_arch.csv'
# # covid_arc = corona_data(url1, url2, file1, 'Y')
# ### csse_covid_19_daily_reports 
# url3 = 'https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports'
# url4 = 'https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_daily_reports/'
# file2 = 'coronavirus_dly.csv'
# covid_new = corona_data(url3, url4, file2, 'Y')
# ### csse_covid_19_daily_reports_us
# url5 = 'https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports_us'
# url6 = 'https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_daily_reports_us/'
# file3 = 'coronavirus_dly_us.csv'
# covid_usa = corona_data(url5, url6, file3, 'Y')

In [21]:
def coviddonwload(filename, url_, mnt_year, save = 'Y', output = ['Y', 'N']):
    ''' 
    Function to download and update local version of covid data from Github into the desired file (filename) 
        - US DATA:
        'https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports_us'
        - WORLD DATA
        'https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports'
    
    PARAMETERS:
        filename = local file to store the data - csv
        url_ = one of the above urls
        mnt_year = string in format MM-YYYY
        save = whether or not to save the local file - useful to access the data fast
        output = whether of not to produce a df as output

    USAGE:
        df_covid_us = coviddonwload('covid_data_github.csv', url, '11-2021', save = 'Y', output = 'Y')
    '''
    # Contents of the local file
    print(mnt_year)
    try:
        fdf = pd.read_csv(filename, low_memory = False)
        current_files = fdf.source.unique()
    except:
        fdf = pd.DataFrame()
        current_files = []
    covgitdata = requests.get(url_)
    soup = BeautifulSoup(covgitdata.content, "lxml")
    
    spans = soup.find_all('span', attrs={'class':'css-truncate'})
    
    spanlist = []
    for sp in spans:
        try: 
            sp = str(sp.string)
        except:
            continue
        if ('csv' in sp)&(mnt_year[-4:] in sp)&(sp.startswith(mnt_year[:2])):
            spanlist.append(sp)
    spanlist = [data for data in spanlist if data not in fdf.source.unique()]
    if len(spanlist) == 0:
        print('No updates found -- No new data added to existing file')
        return 
    else:
        print('{} New updates found!!!'.format(len(spanlist)))
        for file in spanlist:
            df1 = pd.read_html(url_+'/'+file, index_col=0)[0].reset_index(drop = True)
            df1.loc[:,'source'] = file
            fdf = pd.concat([fdf, df1], axis = 0)

        if (save == 'Y'):
            fdf.to_csv(filename, index = False)    
            print('File {} updated'.format(filename))
        if output =='Y':
            return fdf
        else:
            return

In [36]:
urlus = 'https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports_us'
start_mont = 1
end_mont = 1
year = 2023
[coviddonwload('Covid_data/covid_data_github.csv', urlus, dt, save = 'Y', output = 'N') for dt in ['0'+str(d)+'-'+str(year) if len(str(d))==1 else str(d)+'-'+str(year) for d in range(start_mont, end_mont+1,1)]]

01-2023
4 New updates found!!!
File Covid_data/covid_data_github.csv updated


[None]

In [37]:
# Read local dataset
covid_arc = pd.read_csv('Covid_data/covid_data_github.csv', low_memory = False)
corv_us = covid_arc[covid_arc['Country_Region']=='US']
corv_us['Province_State'].unique()

array(['Alabama', 'Alaska', 'American Samoa', 'Arizona', 'Arkansas',
       'California', 'Colorado', 'Connecticut', 'Delaware',
       'Diamond Princess', 'District of Columbia', 'Florida', 'Georgia',
       'Grand Princess', 'Guam', 'Hawaii', 'Idaho', 'Illinois', 'Indiana',
       'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland',
       'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi',
       'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire',
       'New Jersey', 'New Mexico', 'New York', 'North Carolina',
       'North Dakota', 'Northern Mariana Islands', 'Ohio', 'Oklahoma',
       'Oregon', 'Pennsylvania', 'Puerto Rico', 'Rhode Island',
       'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah',
       'Vermont', 'Virgin Islands', 'Virginia', 'Washington',
       'West Virginia', 'Wisconsin', 'Wyoming', 'Recovered'], dtype=object)

In [38]:
# #Getting latitude and longitude from geocode for each state
# from geopy.geocoders import Nominatim
# geolocator = Nominatim(user_agent='myapplication')

# st_dc={}

# for state in corv_us['Province_State'].unique():
#     st_location = geolocator.geocode(state)
#     st_dc[state] = {'lat':st_location.latitude, 'lon':st_location.longitude}
# st_df = pd.DataFrame(st_dc).T.reset_index().rename(columns = {'index':'Province_State'})
# st_df.shape

In [39]:
us_state_abbrev = {'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ','Arkansas': 'AR', 'California': 'CA', 
                    'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE', 'Florida': 'FL','Georgia': 'GA',
                    'Hawaii': 'HI', 'Idaho': 'ID', 'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA',
                    'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD',
                    'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS', 'Missouri': 'MO',
                    'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ',
                    'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND', 'Northern Mariana Islands': 'MP', 'Ohio': 'OH',
                    'Oklahoma': 'OK', 'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC',
                    'South Dakota': 'SD', 'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT',
                    'Virginia': 'VA', 'Washington': 'WA', 'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY' ,          
                    'District of Columbia': 'DC',
                    'American Samoa': 'AS', 'Guam': 'GU', 'Puerto Rico': 'PR', 'Virgin Islands': 'VI',
                    }
state_field = 'Province_State'
corv_us['State'] = np.where(corv_us[state_field].str.contains('|'.join(us_state_abbrev.values())), corv_us[state_field].str.extract(r"\,\s([A-Z]{2})", expand=False)
                , np.where(corv_us[state_field].str.contains(r'Washington, D.C.', case = False), 'DC'
                    , np.where(corv_us[state_field].str.contains(r'Chicago', case = False), 'IL', corv_us[state_field].map(us_state_abbrev)
                        )))
corv_us.head()

,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,FIPS,...,Case_Fatality_Ratio,UID,ISO3,Testing_Rate,Hospitalization_Rate,source,People_Tested,Mortality_Rate,Date,State
0,Alabama,US,2021-12-02 04:33:13,32.3182,-86.9023,846297,16143,NaN,NaN,1.0,...,1.907486,84000001.0,USA,128306.702684,NaN,12-01-2021.csv,NaN,NaN,NaN,AL
1,Alaska,US,2021-12-02 04:33:13,61.3707,-152.4044,151323,881,NaN,NaN,2.0,...,0.582198,84000002.0,USA,482687.326139,NaN,12-01-2021.csv,NaN,NaN,NaN,AK
2,American Samoa,US,2021-12-02 04:33:13,-14.2710,-170.1320,4,0,NaN,NaN,60.0,...,0.000000,16.0,ASM,3846.084722,NaN,12-01-2021.csv,NaN,NaN,NaN,AS
3,Arizona,US,2021-12-02 04:33:13,33.7298,-111.4312,1272943,22350,NaN,NaN,4.0,...,1.755774,84000004.0,USA,200594.321774,NaN,12-01-2021.csv,NaN,NaN,NaN,AZ
4,Arkansas,US,2021-12-02 04:33:13,34.9697,-92.3731,529768,8687,NaN,NaN,5.0,...,1.639774,84000005.0,USA,140262.058106,NaN,12-01-2021.csv,NaN,NaN,NaN,AR


In [40]:
corv_us2 = corv_us[['Country_Region', 'Province_State', 'Last_Update', 'Confirmed','Deaths', 'Recovered', 'People_Hospitalized','source', 'State', 'Lat', 'Long_']]
corv_us2.columns = ['Country', 'State_nm', 'Last_updt', 'Confirmed', 'Deaths', 'Recovered', 'Hospitalizations', 'Source', 'State', 'Lat', 'Lon']
corv_us2 = corv_us2[corv_us2.State.isin([
        'SC', 'LA', 'VA', 'ID', 'IA', 'KY', 'MO', 'OK', 'CO', 'IL', 'IN',
        'MS', 'NE', 'OH', 'PA', 'WA', 'WI', 'VT', 'MN', 'FL', 'NC', 'CA',
        'NY', 'WY', 'MI', 'MD', 'AK', 'KS', 'TN', 'TX', 'ME', 'AZ', 'GA',
        'AR', 'NJ', 'SD', 'AL', 'WV', 'ND', 'MA', 'UT', 'MT', 'NH', 'OR',
        'NM', 'RI', 'NV', 'DC', 'CT', 'HI', 'DE'])]
corv_us2.loc[:,'date']= corv_us2.Source.str.extract(r'([0-9]{2}-[0-9]{2}-[0-9]{4})', expand=False)
corv_us2.head()

,Country,State_nm,Last_updt,Confirmed,Deaths,Recovered,Hospitalizations,Source,State,Lat,Lon,date
0,US,Alabama,2021-12-02 04:33:13,846297,16143,NaN,NaN,12-01-2021.csv,AL,32.3182,-86.9023,12-01-2021
1,US,Alaska,2021-12-02 04:33:13,151323,881,NaN,NaN,12-01-2021.csv,AK,61.3707,-152.4044,12-01-2021
3,US,Arizona,2021-12-02 04:33:13,1272943,22350,NaN,NaN,12-01-2021.csv,AZ,33.7298,-111.4312,12-01-2021
4,US,Arkansas,2021-12-02 04:33:13,529768,8687,NaN,NaN,12-01-2021.csv,AR,34.9697,-92.3731,12-01-2021
5,US,California,2021-12-02 06:32:33,5088361,74528,NaN,NaN,12-01-2021.csv,CA,36.1162,-119.6816,12-01-2021


# MAPS

In [41]:

max_date_file = max([datetime.strptime(re.findall(r'^\d+-\d+-\d+', x)[0], '%m-%d-%Y') for x in corv_us2.Source.unique()]).strftime("%m-%d-%Y")+'.csv'

corv_us_last = corv_us2[corv_us2.Source == max_date_file].reset_index(drop = True)
corv_us_last.head()

c_state = corv_us_last.groupby('State').Confirmed.sum()\
                                    .to_frame().merge(corv_us_last.groupby('State').Lat.mean(), left_index=True, right_index=True)\
                                    .merge(corv_us_last.groupby('State').Lon.mean(), left_index=True, right_index=True)\
                                    .merge(corv_us_last.groupby('State').State_nm.first(), left_index=True, right_index=True).reset_index(drop = False)
c_state.to_csv('Covid_data/coronavirus_latest_data.csv', index = False)

c_state.rename(columns = {'Lat':'Latitude',	'Lon':'Longitude'}, inplace = True)
# Setting the size of the circle based on the number of cases
c_state.loc[:,'scale'] = c_state.Confirmed/max(c_state.Confirmed)
min_size = c_state[c_state.scale >= 0.08].scale.min()
max_size = c_state[c_state.scale >= 0.08].scale.max()

c_state.loc[:,'size'] = np.where(c_state.Confirmed/max(c_state.Confirmed)==1, 100/2,
                            np.where(c_state.Confirmed/max(c_state.Confirmed)>0.08, c_state.Confirmed/max(c_state.Confirmed)*100/2, min_size*100/2))

### US

In [42]:
# # https://jcutrer.com/python/learn-geopandas-plotting-usmaps

# states = gpd.read_file('geopandas-tutorial/data/usa-states-census-2014.shp')
# states.head()
# # “EPSG:4326” WGS84 Latitude/Longitude, used in GPS
# # “EPSG:3395” Spherical Mercator. Google Maps, OpenStreetMap, Bing Maps
# # “EPSG:32633” UTM Zones (North) – (Universal Transverse Mercator)
# # “EPSG:32733” UTM Zones (South) – (Universal Transverse Mercator)
# states.crs
# states = states.to_crs("EPSG:3395")
# states.crs
#states.plot()

#### PLOTS

In [43]:
# Get Geographical data
gdf = gpd.read_file('Covid_data/ne_110m_admin_0_countries.shp')[['ADMIN','ADM0_A3','geometry']]
gdf.columns = ['country', 'country_code', 'geometry']
gdf = gdf[gdf['country_code'] == 'USA']
# print(gdf)
# print('####', gdf.shape)
#Read data to json.
gdf_json = json.loads(gdf.to_json())
# Convert to String like object.
mgrid = json.dumps(gdf_json)
grid_crs=gdf.crs
# print(grid_crs)

def map_plot1(dataf, lat, lng, grid_, key,  zoom=3, map_type='roadmap'):
    
    gmap_options = GMapOptions(lat=lat, lng=lng, 
                               map_type=map_type, zoom=zoom)
    p = gmap(key, gmap_options, title='COVID-19 Cases in the USA by State', 
             width=600, height=400)
    
    geosourc = GeoJSONDataSource(geojson = grid_)
    pointsourc = ColumnDataSource(dataf)

    tile_provider = get_provider(CARTODBPOSITRON)

    #Add patch renderer to figure. 
#     p.patches('xs','ys', source = geosourc,fill_color = 'lightgrey',
#               line_color = 'black', line_width = 0.25, fill_alpha = 0.5)
    f1 = p.circle('Longitude', 'Latitude',source=pointsourc, size='size', fill_alpha = 0.5)
    
    p.add_tools(HoverTool(renderers = [f1],
    tooltips=[ ('', '@State_nm'), ('', '@Confirmed{0,0}'),
             ]))
    return p

In [49]:
#Scattered plots and Layout
data = corv_us2.groupby(['State', 'date']).Confirmed.sum().reset_index().rename(columns = {'State':'state_nm', 'Confirmed':'cases'})
data.loc[:,'date'] = data.date.astype(np.datetime64)

# Select States
available_states = list(set(data['state_nm']))
available_states.sort()
# DataFrame with all the information
alldf = ColumnDataSource(data)
# Empty source so the plot is empty before data is selected and this will be the data shown in the plot
current_1 = ColumnDataSource(dict(data[data.state_nm =='TX']))
current_2 = ColumnDataSource(dict(date = [], cases = [], state_nm =[]))

# view_1 = CDSView(source=current_1, filters=[IndexFilter([i for i in range(data.shape[0]-10, data.shape[0], 1)])])

hover_tool = HoverTool(tooltips = [('State', '@state_nm'),
                                   ('Date',  '@date{%Y-%m-%d}'),
                                   ('Cases', '@cases{0,0}'),
                                  ],
                       formatters = {'@date': 'datetime'})

def make_my_plot(alldfs, width, heigth, tools):
    # view = CDSView(source=alldfs, filters=[IndexFilter([i for i in range(len(alldfs.data['state_nm'])-10, len(alldfs.data['state_nm']), 1)])])
    plot = figure(width = width, height = heigth,  tools = tools)#, x_range = [(alldfs.data['date'].max() - timedelta(days=x)).strftime('%Y-%m-%d') for x in range(10)])# = alldf['date'].max())
    plot.scatter(x = 'date', y = 'cases', source = alldfs
                # , view = view
                )#  color = 'state_color', line_width=2)
    plot.yaxis.formatter=NumeralTickFormatter(format="0,0")
    plot.yaxis.axis_label = 'Cases (#)'
    plot.xaxis.formatter=DatetimeTickFormatter(days="%b %d")
    plot.xaxis.major_label_orientation = pi/4
    plot.xaxis.axis_label = 'Date'
    plot.add_layout(Title(text = 'Cumulative Coronavirus Cases in the USA by State(s)',
                            text_font_size = '12pt', text_font_style = 'italic'), 'above')

    return plot

def call_back_fun(all_data, current_1):
    callb = CustomJS(args = dict(source=all_data, sc=current_1),
                       code = '''
                            var selected_state_nm   = cb_obj.value;
                            sc.data['cases'] = [];
                            sc.data['date'] = [];
                            sc.data['state_nm']= [];
                            for (let i = 0; i < source.data['cases'].length; i++) {
                              if (selected_state_nm.indexOf(source.data['state_nm'][i]) >= 0) {
                                sc.data['cases'][i] = source.data['cases'][i];
                                sc.data['date'][i]  = source.data['date'][i];
                                sc.data['state_nm'][i] = source.data['state_nm'][i];
                                }
                              }
                              console.log(sc.data);
                            sc.change.emit();
                            '''
                    )
    return callb
    
def multi_select(all_data, current_data, select_tittle, size):
    multiselect = MultiSelect(title = select_tittle, value = [], options = available_states, width = 150, size = size)
    multiselect.js_on_change('value', call_back_fun(all_data, current_data))
    return multiselect

width = 500 
height = 300
Select_tittle = 'Select state(s)'
tools = [hover_tool, 'box_zoom, pan, reset']
size_selector = 15
plot1 = make_my_plot(current_1, width, height, tools)

lat, lon = 45, -115
cases_map1 = map_plot1(c_state, lat, lon, mgrid, keys.goggle_gmaps_api)


page_layout1 = column(row(multi_select(alldf, current_1, Select_tittle, size_selector), plot1), cases_map1)
show(grid([page_layout1])) 

In [45]:

corv_us2[corv_us2.State_nm=='Alabama'].sort_values(by = 'Last_updt')

,Country,State_nm,Last_updt,Confirmed,Deaths,Recovered,Hospitalizations,Source,State,Lat,Lon,date
26332,US,Alabama,2020-04-12 23:18:15,3667,93,NaN,437.0,04-12-2020.csv,AL,32.3182,-86.9023,04-12-2020
26391,US,Alabama,2020-04-13 23:07:54,3870,99,NaN,457.0,04-13-2020.csv,AL,32.3182,-86.9023,04-13-2020
26450,US,Alabama,2020-04-14 23:33:31,4041,114,NaN,493.0,04-14-2020.csv,AL,32.3182,-86.9023,04-14-2020
26509,US,Alabama,2020-04-15 22:56:51,4307,118,NaN,525.0,04-15-2020.csv,AL,32.3182,-86.9023,04-15-2020
26568,US,Alabama,2020-04-16 23:30:51,4465,133,NaN,553.0,04-16-2020.csv,AL,32.3182,-86.9023,04-16-2020
...,...,...,...,...,...,...,...,...,...,...,...,...
57612,US,Alabama,2023-01-01 04:31:23,1568934,20737,NaN,NaN,12-31-2022.csv,AL,32.3182,-86.9023,12-31-2022
57670,US,Alabama,2023-01-02 04:31:21,1568934,20737,NaN,NaN,01-01-2023.csv,AL,32.3182,-86.9023,01-01-2023
57728,US,Alabama,2023-01-03 04:31:34,1568934,20737,NaN,NaN,01-02-2023.csv,AL,32.3182,-86.9023,01-02-2023
57786,US,Alabama,2023-01-04 04:31:12,1568934,20737,NaN,NaN,01-03-2023.csv,AL,32.3182,-86.9023,01-03-2023


In [46]:
corv_us[corv_us.Province_State=='Alabama'].sort_values(by = 'Last_Update')

,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,FIPS,...,Case_Fatality_Ratio,UID,ISO3,Testing_Rate,Hospitalization_Rate,source,People_Tested,Mortality_Rate,Date,State
26332,Alabama,US,2020-04-12 23:18:15,32.3182,-86.9023,3667,93,NaN,NaN,1.0,...,NaN,84000001.0,USA,1265.075660,12.264945,04-12-2020.csv,62029.0,2.536133,NaN,AL
26391,Alabama,US,2020-04-13 23:07:54,32.3182,-86.9023,3870,99,NaN,NaN,1.0,...,NaN,84000001.0,USA,1318.245181,12.238886,04-13-2020.csv,64636.0,2.558140,NaN,AL
26450,Alabama,US,2020-04-14 23:33:31,32.3182,-86.9023,4041,114,NaN,NaN,1.0,...,NaN,84000001.0,USA,1377.818704,12.471541,04-14-2020.csv,67557.0,2.821084,NaN,AL
26509,Alabama,US,2020-04-15 22:56:51,32.3182,-86.9023,4307,118,NaN,NaN,1.0,...,NaN,84000001.0,USA,1444.428468,12.883436,04-15-2020.csv,70823.0,2.739726,NaN,AL
26568,Alabama,US,2020-04-16 23:30:51,32.3182,-86.9023,4465,133,NaN,NaN,1.0,...,NaN,84000001.0,USA,1506.734908,12.727273,04-16-2020.csv,73878.0,2.978723,NaN,AL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57612,Alabama,US,2023-01-01 04:31:23,32.3182,-86.9023,1568934,20737,NaN,NaN,1.0,...,1.321725,84000001.0,USA,NaN,NaN,12-31-2022.csv,NaN,NaN,NaN,AL
57670,Alabama,US,2023-01-02 04:31:21,32.3182,-86.9023,1568934,20737,NaN,NaN,1.0,...,1.321725,84000001.0,USA,NaN,NaN,01-01-2023.csv,NaN,NaN,NaN,AL
57728,Alabama,US,2023-01-03 04:31:34,32.3182,-86.9023,1568934,20737,NaN,NaN,1.0,...,1.321725,84000001.0,USA,NaN,NaN,01-02-2023.csv,NaN,NaN,NaN,AL
57786,Alabama,US,2023-01-04 04:31:12,32.3182,-86.9023,1568934,20737,NaN,NaN,1.0,...,1.321725,84000001.0,USA,NaN,NaN,01-03-2023.csv,NaN,NaN,NaN,AL


In [47]:
corv_us2.sort_values(by = ['State_nm', 'date'], inplace = True)
corv_us2.loc[:,'daily'] = corv_us2.groupby(['State_nm'])['Confirmed'].diff()
corv_us2[corv_us2.State_nm=='Alaska'].sort_values(by = 'Last_updt')

,Country,State_nm,Last_updt,Confirmed,Deaths,Recovered,Hospitalizations,Source,State,Lat,Lon,date,daily
26333,US,Alaska,2020-04-12 23:18:15,272,8,66.0,31.0,04-12-2020.csv,AK,61.3707,-152.4044,04-12-2020,-248403.0
26392,US,Alaska,2020-04-13 23:07:54,277,8,85.0,32.0,04-13-2020.csv,AK,61.3707,-152.4044,04-13-2020,-248398.0
26451,US,Alaska,2020-04-14 23:33:31,285,9,98.0,32.0,04-14-2020.csv,AK,61.3707,-152.4044,04-14-2020,-249685.0
26510,US,Alaska,2020-04-15 22:56:51,293,9,106.0,34.0,04-15-2020.csv,AK,61.3707,-152.4044,04-15-2020,-249685.0
26569,US,Alaska,2020-04-16 23:30:51,300,9,110.0,35.0,04-16-2020.csv,AK,61.3707,-152.4044,04-16-2020,-249678.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
57613,US,Alaska,2023-01-01 04:31:23,301432,1455,NaN,NaN,12-31-2022.csv,AK,61.3707,-152.4044,12-31-2022,144263.0
57671,US,Alaska,2023-01-02 04:31:21,301432,1455,NaN,NaN,01-01-2023.csv,AK,61.3707,-152.4044,01-01-2023,144263.0
57729,US,Alaska,2023-01-03 04:31:34,301432,1455,NaN,NaN,01-02-2023.csv,AK,61.3707,-152.4044,01-02-2023,144263.0
57787,US,Alaska,2023-01-04 04:31:12,301803,1455,NaN,NaN,01-03-2023.csv,AK,61.3707,-152.4044,01-03-2023,141714.0


In [48]:
data.loc[:,'daily'] = data.sort_values(by = ['state_nm', 'date']).groupby(['state_nm'])['cases'].diff()
data[data.state_nm=='AL'].sort_values(by = 'date')



,state_nm,date,cases,daily
1204,AL,2020-04-12,3667,NaN
1207,AL,2020-04-13,3870,203.0
1210,AL,2020-04-14,4041,171.0
1213,AL,2020-04-15,4307,266.0
1216,AL,2020-04-16,4465,158.0
...,...,...,...,...
1995,AL,2022-12-31,1568934,0.0
1000,AL,2023-01-01,1568934,0.0
1003,AL,2023-01-02,1568934,0.0
1006,AL,2023-01-03,1568934,0.0


In [35]:
corv_us2[corv_us2.State_nm=='Alaska'].sort_values(by = 'Last_updt')

,Country,State_nm,Last_updt,Confirmed,Deaths,Recovered,Hospitalizations,Source,State,Lat,Lon,date,daily
26333,US,Alaska,2020-04-12 23:18:15,272,8,66.0,31.0,04-12-2020.csv,AK,61.3707,-152.4044,04-12-2020,-248403.0
26392,US,Alaska,2020-04-13 23:07:54,277,8,85.0,32.0,04-13-2020.csv,AK,61.3707,-152.4044,04-13-2020,-248398.0
26451,US,Alaska,2020-04-14 23:33:31,285,9,98.0,32.0,04-14-2020.csv,AK,61.3707,-152.4044,04-14-2020,-249685.0
26510,US,Alaska,2020-04-15 22:56:51,293,9,106.0,34.0,04-15-2020.csv,AK,61.3707,-152.4044,04-15-2020,-249685.0
26569,US,Alaska,2020-04-16 23:30:51,300,9,110.0,35.0,04-16-2020.csv,AK,61.3707,-152.4044,04-16-2020,-249678.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
57381,US,Alaska,2022-12-28 04:31:43,301432,1455,NaN,NaN,12-27-2022.csv,AK,61.3707,-152.4044,12-27-2022,145255.0
57439,US,Alaska,2022-12-29 04:31:52,301432,1455,NaN,NaN,12-28-2022.csv,AK,61.3707,-152.4044,12-28-2022,145255.0
57497,US,Alaska,2022-12-30 04:32:02,301432,1455,NaN,NaN,12-29-2022.csv,AK,61.3707,-152.4044,12-29-2022,144263.0
57555,US,Alaska,2022-12-31 04:32:10,301432,1455,NaN,NaN,12-30-2022.csv,AK,61.3707,-152.4044,12-30-2022,144263.0
